# Tarea 1

Fecha de envío del PR inicial: **26 de febrero**

Fecha de aceptación del PR: **12 de marzo**


## 1. Raíces de una ecuación de segundo grado

Implementen en julia una función que devuelva las raíces de una ecuación de segundo grado general con coeficientes reales. La función debe ser estable según el tipo ([type-stable](http://docs.julialang.org/en/release-0.5/manual/faq/?highlight=type%20stability#what-does-type-stable-mean)), lo que significa que el tipo del resultado sólamente depende del tipo de las entradas. En particular, muestren que su función devuelve raíces consistentes para $a=1$, $b=1$, $c=10^{-17}$.

La función debe estar documentada de manera apropiada, lo que significa que, al correr el Jupyter notebook de su tarea, la instrucción
```julia
? nombre_funcion
```
debe dar información apropiada.

Incluyan cualquier comentario prudente del algoritmo que usan, ya sea en el notebook (usando markdown) o en la propia función incluyendo comentarios.


In [ ]:
function raices(a,b,c)
    det = b^2 - 4 * a * c
    
    if det < 0
        println(det)
        println("La funcion no tiene raices reales")
    elseif det == 0
        println("pepe")
        println(det)
        return -b / 2 * a
    else
        println("pollo")
        println(det)
        return (-b + sqrt(b ^ 2 - 4 * a* c)) / (2 * a), (-b - sqrt(b ^ 2 - 4 * a* c)) / (2 * a)

    end
end

In [ ]:
raices(1,1,0.000001)

## 2. Promedio y varianza

(a) Escriban dos funciones (`promedio` y `varianza`) que calculen el promedio y la varianza de un vector de datos. 
    
(b) Comparen su resultado con lo que se obtiene al utilizar `mean` y `var`, que son las funciones nativas de Julia que hacen eso. En particular, consideren el siguiente vector de datos
```julia
v = 1.0e7 + randn(10^8)
```
¿Qué hace la función `randn(10^8)`?

(c) Implementen el [algoritmo recursivo](https://en.wikipedia.org/wiki/Algorithms_for_calculating_variance#Online_algorithm), ampliamente desconocido, que  calcula el promedio (`promedio_rec`) y la varianza (`varianza_rec`) con una sóla lectura de los datos. Usen sus fuciones en el vector anterior y comparen sus resultados con los obtenidos en (a).

(d) ¿Cuál de todos los algoritmos es el más rápido? (Pueden usar `@time` o `@elapsed`.) Usen vectores de varias longitudes, que cubran varios órdenes de magnitud.

v es un arreglo con 10^8 entradas con un valor aleatorio alrrededor de 1.0e7

In [6]:
function promedio(vector)
    n = length(vector)
    
    num = sum(vector)
    
    return num / n
    
end

promedio (generic function with 1 method)

In [91]:
v = 1.0e7 + randn(10^4)
p = promedio(v)

9.999999992861398e6

In [80]:
function varianza(vector, promedio)
    n = length(vector)
    for i = vector
        sum = (i - promedio)^2
    end
    sum = sum / n
    
    return sqrt(sum)
        
end

varianza (generic function with 1 method)

In [92]:
varianza(v, p)

0.01978055587038398

## 3. Método de Simpson para integrar $f(x)$ en un intervalo dado

Implementen el método de Simpson para integrar una función arbitraria $f(x)$.